In [1]:
# # header
import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [2]:
# # imports
import torch
import os
from pprint import pformat
from src.composer import COCOTokens, COCOAnimals, BG20k
from src.composer import PerceptualGrouping_COCO, Recognition_COCO, Search_COCO, SearchGrid_COCO
from src.conductor import AttentionTrain
from src.model import AttentionModel
from src.utils import plot_all
from src.utils import build_loaders, get_n_parameters
from prelude import get_device, startup_folders, load_dicts

In [3]:
start_folder = r"/Users/saeedida/GitProjects/bio-attention/pretrained/coco"
results_folder, logger = startup_folders(start_folder, name=f"exp_coco")
data_path = r"../data"

/Users/saeedida/GitProjects/bio-attention/pretrained/coco/1728903725 was created!


In [4]:
model_params = load_dicts(start_folder, "model_params")
tasks = load_dicts(start_folder, "tasks")
train_params = load_dicts(start_folder, "train_params")
DeVice, num_workers, pin_memory = get_device()
print(f"model_params: {model_params}")
print(f"tasks: {tasks}")
print(f"train_params: {train_params}")

Device set to mps
model_params: {'in_dims': [3, 256, 256], 'n_classes': 10, 'out_dim': 20, 'normalize': True, 'softness': [0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0], 'channels': [3, 32, 32, 64, 64, 128, 128, 256, 256], 'residuals': False, 'kernels': [7, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'strides': 1, 'paddings': 'same', 'conv_bias': True, 'conv_norms': [None, 'layer', 'layer', 'layer', 'layer', 'layer', 'layer', 'layer'], 'conv_dropouts': 0.0, 'conv_funs': GELU(approximate='none'), 'deconv_funs': GELU(approximate='none'), 'deconv_norms': [None, 'layer', 'layer', 'layer', 'layer', 'layer', 'layer', 'layer'], 'pools': 2, 'rnn_dims': [256], 'rnn_bias': True, 'rnn_dropouts': 0.0, 'rnn_funs': GELU(approximate='none'), 'n_tasks': 3, 'task_layers': 1, 'task_weight': True, 'task_bias': True, 'task_funs': Tanh(), 'rnn_to_fc': False, 'trans_fun': Identity(), 'affine': False}
tasks: {'Recognition': {'composer': 'Recognition_COCO', 'key': 0, 'params': {'n_iter': 3, 'stride': 64, 'blank': False, 'static': 

In [5]:
tasks['Recognition']["composer"] = Recognition_COCO
tasks['PerceptualGrouping']["composer"] = PerceptualGrouping_COCO
tasks['Search']["composer"] = Search_COCO
tasks['SearchGrid']["composer"] = SearchGrid_COCO

tasks['Recognition']["datasets"] = []
tasks['PerceptualGrouping']["datasets"] = []
tasks['Search']["datasets"] = []
tasks['SearchGrid']["datasets"] = []

tasks['Recognition']["dataloaders"] = []
tasks['PerceptualGrouping']["dataloaders"] = []
tasks['Search']["dataloaders"] = []
tasks['SearchGrid']["dataloaders"] = []

tasks['Recognition']["loss_s"] = (slice(0, None, None), slice(0, None, None))
tasks['PerceptualGrouping']["loss_s"] = (slice(1, None, None), slice(1, None, None))
tasks['Search']["loss_s"] = (slice(1, None, None), slice(1, None, None))
tasks['SearchGrid']["loss_s"] = (slice(1, None, None), slice(1, None, None))

tasks['Recognition']["loss_s"] = ((-1, ), (-1, ))
tasks['PerceptualGrouping']["loss_s"] = ((-1, ), (-1, ))
tasks['Search']["loss_s"] = ((-1, ), (-1, ))
tasks['SearchGrid']["loss_s"] = ((-1, ), (-1, ))

In [6]:
# datasets and dataloaders
coco_tokens = COCOTokens(directory=data_path, animals=True, split=0.9)
train_tks, valid_tks, test_tks = coco_tokens.get_tokens()
train_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=0, tokens=train_tks)
valid_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=1, tokens=valid_tks)
test_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=2, tokens=test_tks)
train_bg = BG20k(root=data_path, kind="train")
test_bg = valid_bg = BG20k(root=data_path, kind="test")
DeVice, num_workers, pin_memory = get_device()
for o in tasks:
    if tasks[o]["composer"] in (Recognition_COCO , SearchGrid_COCO):
        tasks[o]["datasets"].append(tasks[o]["composer"](train_coco, **tasks[o]["params"], bg_dataset=train_bg))
        tasks[o]["datasets"].append(tasks[o]["composer"](valid_coco, **tasks[o]["params"], bg_dataset=valid_bg))
        tasks[o]["datasets"].append(tasks[o]["composer"](test_coco, **tasks[o]["params"], bg_dataset=test_bg))
    else:
        tasks[o]["datasets"].append(tasks[o]["composer"](train_coco, **tasks[o]["params"]))
        tasks[o]["datasets"].append(tasks[o]["composer"](valid_coco, **tasks[o]["params"]))
        tasks[o]["datasets"].append(tasks[o]["composer"](test_coco, **tasks[o]["params"]))
    tasks[o]["datasets"][1].build_valid_test()
    tasks[o]["datasets"][2].build_valid_test()
    tasks[o]["dataloaders"] = build_loaders(tasks[o]["datasets"], batch_size=train_params["batch_size"], num_workers=num_workers, pin_memory=pin_memory, shuffle=False)
assert model_params["n_classes"] == train_coco.n_classes, f"Number of n_classes {model_params['n_classes']} and n_classes {train_coco.n_classes} must be equal!"
tasks["PerceptualGrouping"]["class_weights"] = train_coco.class_weights if hasattr(train_coco, "class_weights") else None
tasks["Recognition"]["class_weights"] = train_coco.class_weights if hasattr(train_coco, "class_weights") else None


loading annotations into memory...
Done (t=9.49s)
creating index...
index created!
loading annotations into memory...
Done (t=1.56s)
creating index...
index created!
loading annotations into memory...
Done (t=11.77s)
creating index...
index created!
loading annotations into memory...
Done (t=15.53s)
creating index...
index created!
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
Device set to mps


In [7]:
# model and optimizer...
model = AttentionModel(**model_params)
logger.info(model)
logger.info(f"Model has {get_n_parameters(model):,} parameters!")
optimizer = torch.optim.Adam(model.parameters(), lr=train_params["lr"])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=train_params["milestones"], gamma=train_params["gamma"])
model_trainer = AttentionTrain(model, optimizer, scheduler, tasks, logger, results_folder)

# load model
model_dir = os.path.join(start_folder, "model" + ".pth")
assert os.path.exists(model_dir), "Could not find the model.pth in the given dir!"
model.load_state_dict(torch.load(model_dir, map_location=DeVice))

AttentionModel(
  (task_funs): Tanh()
  (trans_fun): Identity()
  (conv_blocks): ModuleList(
    (0): ConvBlock(
      (conv): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
      (norm): Identity()
      (fun): GELU(approximate='none')
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (norm): GroupNorm(1, 32, eps=1e-05, affine=False)
      (fun): GELU(approximate='none')
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvBlock(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (norm): GroupNorm(1, 64, eps=1e-05, affine=False)
      (fun): GELU(approximate='none')
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3

<All keys matched successfully>

In [8]:
# plotting
plot_all(10, model, tasks, results_folder, "_test", DeVice, logger, False, "test")


In [9]:
# evaluating...
model_trainer.eval(DeVice, "test", False)


testing...
  Task Recognition:
    CEi Loss: 0.055770    CEe Loss: 0.056129    Pix Err: 0.029281    Att Acc: 0.916078    Cls Acc: 1233/1558
  Task PerceptualGrouping:
    CEi Loss: 0.066535    CEe Loss: 0.065844    Pix Err: 0.028895    Att Acc: 0.858874    Cls Acc: 1179/1558
  Task Search:
    CEi Loss: 0.714598    CEe Loss: 0.676482    Pix Err: 0.035254    Att Acc: 0.872138    Cls Acc: 482/646
  Task SearchGrid:
    CEi Loss: 0.551441    CEe Loss: 0.535923    Pix Err: 0.023674    Att Acc: 0.880425    Cls Acc: 1282/1558
